# SNOMED

SNOMED CT is a standarised clinical terminology consisting of >350,000 unique concepts. It is owned, maintained and distributed by SNOMED International.

Please visit and explore https://www.snomed.org/ to find out further information about the various SNOMED CT products and services which they offer.

--------

## What is SNOMED CT?

SNOMED CT is a clinical terminology containing concepts with unique meanings and formal logic based definitions organised into hierarchies. For further information please see: https://confluence.ihtsdotools.org/display/DOCSTART/4.+SNOMED+CT+Basics

## SNOMED CT Design
SNOMED CT content is represented into 3 main types of components:

- Concepts representing clinical meanings that are organised into hierarchies.
- Descriptions which link appropriate human readable terms to concepts
- Relationships which link each concept to other related concepts

It also contains mappings to classification systems such as:
- ICD (International classifications of diseases)
- OPCS (Office of Population Censuses and Surveys) (SNOMED UK extension only)

---------

<a name="access_to_snomed_ct"></a>
## Access to SNOMED CT release files

You may download SNOMED CT at the Member country’s designated website. The use of SNOMED CT in Member countries is free. Follow this [link](https://www.snomed.org/our-stakeholders/members) to find out if your country is a member state and explore the website to find directions to where to your national SNOMED CT distribution is held.

E.g. 
* UK -> [NHS TRUD](https://isd.digital.nhs.uk/trud3/user/guest/group/0/home)

* US -> [NIH National Library of Medicine](https://www.nlm.nih.gov/healthit/snomedct/international.html) Alternative clinical terminologies such as UMLS can be found here.


The following Steps are to services provided by SNOMED International for organizations and individuals to request use and access to the International Release of SNOMED CT for use in non-Member countries

__To access SNOMED CT files from non-member contries:__

1.   Please visit the SNOMED [Member Licensing and Distribution Service.](https://mlds.ihtsdotools.org/#/landing) and read their terms and conditions for use.

2.   Login or Register for an account and wait to be granted access.

3.   Once you have been granted access. Logged in and visit the tab ["Release Packages"](https://mlds.ihtsdotools.org/#/viewReleases) and retrieve the release of SNOMED CT that you would like to have. Alternatively, for the international SNOMED release simply visit the [International releases](https://mlds.ihtsdotools.org/#/viewReleases/viewRelease/167).

----------

# Part 1: Preprocessing SNOMED CT for MedCAT

Once you have downloaded a SNOMED release of interest. Store the zipped folder containing your respective SNOMED release in the current colab working directory.

The folder name should look like: `SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z.zip
`


### Install and import required packages

In [ ]:
# Install medcat
!pip install --upgrade medcat

In [ ]:
from medcat.utils.preprocess_snomed import Snomed

### Load the data
Please see the section: [Access to SNOMED CT release files](#access_to_snomed_ct) for how to retrieve the zipped SNOMED CT release.

In [ ]:
# Assign a path to the zipped SNOMED CT release download.
snomed_path = "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z.zip"  # Enter your zipped Snomed folder here

In [ ]:
!unzip '/content/{snomed_path}' -d snomed_folder

### Preprocess the release for MedCAT

In [ ]:
# Initialise
snomed = Snomed("snomed_folder")

In [ ]:
### Skip this step if your version of snomed is not the UK extension released >2021.
### Note: this step will only work with MedCAT v1.2.7+

# snomed.uk_ext = True

#### Create a SNOMED DataFrame

We first preprocess SNOMED to fit the following format:


|cui|name|ontologies|name_status|description_type_ids|type_ids|
|--|--|--|:--:|:--:|--|
|101009|Quilonia ethiopica (organism)|SNOMED|P|organism|81102976|
.
.
.

`cui` - The concept unique identifier, this is simply the `SCTID`.

`name` - This include the name of the concept. The status of the name is given in `name_status`

`ontologies` - Always SNOMED. Alternatively you can change it to your specific edition.

`name_status` - The Fully specified name or FSN is denoted with a `P` - Primary Name. Each concept must be assigned only one Primary Name. These should be unique across all SCTID/cui to avoid confusion. A synonym or other description type is represented as a `A` - Alternative Name. This can be enriched with all possible names and abbreviations for a concept of interest.

`description_type_ids` - These are processed to be the Semantic Tags of the concept.

`type_ids` - This is simply a 10 digit Hash of the Semantic Tags




In [ ]:
# Create SNOMED DataFrame 
df = snomed.to_concept_df()

In [ ]:
df.head()

,cui,name,name_status,ontologies,description_type_ids,type_ids
0,101009,Quilonia ethiopica (organism),P,SNOMED-CT,organism,81102976
1,102002,Hemoglobin Okaloosa (substance),P,SNOMED-CT,substance,91187746
2,103007,Squirrel fibroma virus (organism),P,SNOMED-CT,organism,81102976
3,104001,Excision of lesion of patella (procedure),P,SNOMED-CT,procedure,28321150
4,106004,Structure of posterior carpal region (body str...,P,SNOMED-CT,body structure,37552161


In [ ]:
# inspect
df[df['cui'] == '101009']

,cui,name,name_status,ontologies,description_type_ids,type_ids
0,101009,Quilonia ethiopica (organism),P,SNOMED-CT,organism,81102976
354448,101009,Quilonia ethiopica,A,SNOMED-CT,organism,81102976


In [ ]:
# Optional - Create a SCTID to FSN dictionary
primary_names_only = df[df["name_status"] == 'P']
sctid2name = dict(zip(primary_names_only['cui'], primary_names_only['name']))
del primary_names_only

In [ ]:
sctid2name['101009']

'Quilonia ethiopica (organism)'

#### SNOMED Relationships

In [ ]:
all_snomed_relationships = snomed.list_all_relationships()

In [ ]:
# List of the SCTID of all snomed relationships
all_snomed_relationships

In [ ]:
# Using the SCTID to name to inspect what the FSN (fully specified names) are:
for sctid in all_snomed_relationships:
    print(sctid2name[sctid])

In [ ]:
# save a specific relationship to json
# In the example we save the "IS a (attribute)" hierarchical relationship.
snomed.relationship2json("116680003", "ISA_relationship.json")

#### Mappings to inbuilt external terminologies 

Create a dictionary map to add to the medcat concept database additional information

##### ICD-10
For SNOMED to ICD-10 mapping read more on:
Map Blocks, Map Groups and Map Priorities, for correct official mapping methodology.

In [ ]:
# ICD-10
icd_df = snomed.map_snomed2icd10()

In [ ]:
icd_df.head()

,id,effectiveTime,active,moduleId,refsetId,referencedComponentId,mapGroup,mapPriority,mapRule,mapAdvice,mapTarget,correlationId,mapCategoryId
0,49b07dca-0559-59d9-8c85-3198c941a813,20130731,1,449080006,447562003,10000006,1,1,TRUE,ALWAYS R07.4,R07.4,447561005,447637006
1,2c141272-bbc0-5c34-aa48-72f2dc370ead,20150131,1,449080006,447562003,10001005,1,1,TRUE,ALWAYS A41.9,A41.9,447561005,447637006
2,47cf847c-c85c-5979-8129-98475d4ab274,20190731,1,449080006,447562003,10007009,1,1,TRUE,ALWAYS Q87.1 | POSSIBLE REQUIREMENT FOR ADDITI...,Q87.1,447561005,447637006
3,4cd08e27-0417-5c5d-8873-6a72d130e918,20140731,1,449080006,447562003,1001000119102,1,1,TRUE,ALWAYS I26.9 | MAPPED FOLLOWING WHO GUIDANCE,I26.9,447561005,447637006
4,ca792560-1589-5f24-9126-b040e8ea2b2d,20180731,1,449080006,447562003,10017004,1,1,TRUE,ALWAYS K03.0,K03.0,447561005,447637006


In [ ]:
sctid2icd10 = {k: g["mapTarget"].tolist() for k,g in icd_df[['referencedComponentId',
                                                              'mapTarget']].groupby("referencedComponentId")}

In [ ]:
# To view the SNOMED to ICD-10 Map structure
sctid2icd10

['G40.9', 'G93.8']

##### OPCS
Office of Population Censuses and Surveys


__Note:__ only the SNOMED UK extension edition contains this information
Skip if your version is not a UK extension

In [ ]:
opcs_df = snomed.map_snomed2opcs4()

In [ ]:
opcs_df.head()

### Save for MedCAT

In [ ]:
# Save to CSV for medcat CDB creation
df.to_csv("preprocessed_snomed.csv", index=False)

--------

# Part 2: Create a MedCAT CDB using SNOMED CT release files


These steps are also convered in the tutorial: [Part 3.1 Building a Concept Database and Vocabulary](https://colab.research.google.com/drive/1s1QXJ2E76sZLm5P0Lremw8-kWWXxX_w2#scrollTo=KByaUPYNk7gk)

In [ ]:
# Get the scispacy model.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_md-0.4.0.tar.gz

     |████████████████████████████████| 125.2 MB 33 kB/s 
  Created wheel for en-core-sci-md: filename=en_core_sci_md-0.4.0-py3-none-any.whl size=125733930 sha256=2b5ae33ca2617bc5d309dcdaf1cbb3308698d655892465353e2655eb07536370
  Stored in directory: /root/.cache/pip/wheels/43/54/a8/27fc51c81ba9feda7ba7f9031eb81e33429840cd79d141c88c
Successfully built en-core-sci-md


**Restart the runtime if on colab, sometimes necessary after installing models**

In [ ]:
# Import required packages
from medcat.cdb import CDB
from medcat.config import Config
from medcat.cdb_maker import CDBMaker

#### Create concept database (cdb)

In [ ]:
# First initialise the default configuration
config = Config()
config.general['spacy_model'] = 'en_core_sci_md'
maker = CDBMaker(config)

In [ ]:
# Create an array containing CSV files that will be used to build our CDB
csv_path = ['preprocessed_snomed.csv']

# Create your CDB
cdb = maker.prepare_csvs(csv_path, full_build=True)

Started importing concepts from: preprocessed_snomed.csv
Current progress: 0% at 0.000s per 9231 rows
Current progress: 1% at 55.210s per 9231 rows
Current progress: 2% at 53.859s per 9231 rows
Current progress: 3% at 53.306s per 9231 rows
Current progress: 4% at 52.994s per 9231 rows
Current progress: 5% at 53.183s per 9231 rows
Current progress: 6% at 52.772s per 9231 rows
Current progress: 7% at 53.400s per 9231 rows
Current progress: 8% at 53.544s per 9231 rows
Current progress: 9% at 53.441s per 9231 rows
Current progress: 10% at 53.373s per 9231 rows
Current progress: 11% at 53.139s per 9231 rows
Current progress: 12% at 53.432s per 9231 rows
Current progress: 13% at 54.119s per 9231 rows
Current progress: 14% at 57.038s per 9231 rows
Current progress: 15% at 52.765s per 9231 rows
Current progress: 16% at 53.870s per 9231 rows
Current progress: 17% at 52.843s per 9231 rows
Current progress: 18% at 52.480s per 9231 rows
Current progress: 19% at 53.725s per 9231 rows
Current progre

### Inspect your cdb

In [ ]:
print(cdb.name2cuis['epilepsy'])

['84757009']


In [ ]:
print(cdb.cui2preferred_name['84757009'])

Epilepsy


In [ ]:
print(cdb.cui2names['84757009'])

{'ep~epilepsy', 'epileptic~disorder', 'epileptic', 'epilepsy~disorder', 'epilepsy'}


#### Enrich with extra information and mapping

Mapping was created in [Mappings to inbuilt external terminologies](https://colab.research.google.com/drive/1yesqjMQwQH20Kl9w7siRGVaSWU0uI84W#scrollTo=Mappings_to_inbuilt_external_terminologies).
Here we use [ICD-10](https://colab.research.google.com/drive/1yesqjMQwQH20Kl9w7siRGVaSWU0uI84W#scrollTo=ICD_10) as an example.

In [ ]:
cdb.addl_info['cui2icd10'] = sctid2icd10

### Save your new SNOMED cdb

__tip:__ good practise to include the snomed release edition file name

In [ ]:
cdb.save("SNOMED_cdb.dat")